### Run notebook 01_LoL_predictor_EDA_preparation before running this one, as it prepares the data!

In [194]:
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import yaml

In [195]:
with open('config.yml', mode="r") as f:
    config = yaml.safe_load(f)

import wandb
wandb.login()

wandb.init(
        project="lol-predictor",
        entity="adamzak",
        config=config['train'],
        group='pytorch',
        mode='online'
    )

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


accuracy,▁▁▁▂▂▂▂▃▃▃▄▃▄▅▅▅▅▅▅▅▅▆▆▅▆▆▆▆▆▇▇▇▇██▇█▆█▇
epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
loss,█▇▇▇▇▇▆▆▆▆▅▅▅▅▄▄▄▄▄▄▄▄▃▃▃▃▃▃▃▂▂▂▂▁▁▂▁▂▁▂
accuracy,85.75363
epoch,1999
loss,0.30797


In [196]:
pd.set_option('display.max_columns', 500)
PREPROCESSED_PATH = '../data/processed_high_diamond.csv'

data = pd.read_csv(PREPROCESSED_PATH)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [197]:
data.drop(columns={"gameId"}, inplace=True)

In [198]:
# split data to train and test by 70/30 ratio
train, test = train_test_split(data, test_size=0.3)

# further split test part to 50/50 with validation - final split will be 70/15/15
test, validation = train_test_split(test, test_size=0.5)

In [199]:
train.shape

(6535, 17)

In [200]:
test.shape

(1401, 17)

In [201]:
validation.shape

(1401, 17)

In [202]:
trainX = train.drop(columns={"blueWins"})
trainy = train.blueWins

In [203]:
testX = test.drop(columns={"blueWins"})
testy = test.blueWins

In [204]:
nHidden = config['train']['inputNeurons']

model = nn.Sequential(
    nn.Linear(16, nHidden[0]),
    nn.ReLU(),
    nn.Linear(nHidden[0], nHidden[1]),
    nn.ReLU(),
    nn.Linear(nHidden[1], nHidden[2]),
    nn.ReLU(),
    nn.Linear(nHidden[2], nHidden[3]),
    nn.ReLU(),
    nn.Linear(nHidden[3], 1),
    nn.Sigmoid()
    # Softmax
)

In [205]:
trainX = torch.tensor(trainX.values)
testX = torch.tensor(testX.values)

In [206]:
trainy = torch.tensor(trainy.values)
testy = torch.tensor(testy.values)

# https://stackoverflow.com/a/60440460/12342419
trainy = trainy.type(torch.LongTensor)
testy = testy.type(torch.LongTensor)

In [207]:
def compute_loss(y_hat, y):
    return nn.BCELoss()(y_hat, y)

In [208]:
# Calculate accuracy (a classification metric)
def accuracy_fn(y_true, y_pred):
    correct = torch.eq(y_true, y_pred).sum().item() # torch.eq() calculates where two tensors are equal
    acc = (correct / len(y_pred)) * 100
    return acc

In [209]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [210]:
trainX = trainX.to(torch.float32).to(device)
testX = testX.to(torch.float32).to(device)
trainy = trainy.to(torch.float32).to(device)
testy = testy.to(torch.float32).to(device)

In [211]:
trainy = trainy.reshape((trainy.shape[0], 1))

In [212]:
model

Sequential(
  (0): Linear(in_features=16, out_features=256, bias=True)
  (1): ReLU()
  (2): Linear(in_features=256, out_features=128, bias=True)
  (3): ReLU()
  (4): Linear(in_features=128, out_features=64, bias=True)
  (5): ReLU()
  (6): Linear(in_features=64, out_features=32, bias=True)
  (7): ReLU()
  (8): Linear(in_features=32, out_features=1, bias=True)
  (9): Sigmoid()
)

In [213]:
model.to(device)
print(device)

cuda:0


In [214]:
torch.cuda.is_available()

True

In [215]:
torch.cuda.is_available()
torch.cuda.get_device_name(0)

'NVIDIA GeForce GTX 1050'

In [216]:
num_epochs = config['train']['epochs']

for n in range(num_epochs):
    model.train()
    y_pred = model(trainX)
    loss = compute_loss(y_pred, trainy)
    
    if n % (num_epochs / 100) == 0:
        print(n)
        #print(y_pred)
        print(accuracy_fn(y_true=trainy, y_pred=torch.round(y_pred)))
        accuracy = accuracy_fn(y_true=trainy, y_pred=torch.round(y_pred))
        wandb.log({'epoch': n, 'accuracy': accuracy, 'loss': loss})

    optimizer.zero_grad()
    loss.backward()
    #loss.backward(retain_graph=False)
    optimizer.step()
print(loss)

0
50.16067329762816
100
72.89977046671767
200
72.88446824789594
300
73.06809487375669
400
74.21576128538638
500
74.82785003825555
600
74.91966335118592
700
76.51109410864575
800
76.61820964039786
900
77.12318286151492
1000
77.30680948737567
1100
77.9035960214231
1200
78.62280030604437
1300
78.6381025248661
1400
78.69931140015302
1500
79.43381790359602
1600
79.93879112471308
1700
80.26013771996939
1800
80.87222647283856
1900
81.07115531752103
2000
80.0
2100
81.33129303749044
2200
81.51491966335118
2300
81.28538638102525
2400
81.82096403978576
2500
81.63733741392501
2600
82.18821729150727
2700
82.11170619739863
2800
82.21882172915073
2900
82.61667941851569
3000
82.11170619739863
3100
81.30068859984698
3200
82.5707727620505
3300
82.3106350420811
3400
82.84621270084162
3500
82.95332823259372
3600
83.35118592195869
3700
83.45830145371079
3800
82.78500382555471
3900
83.06044376434582
4000
83.42769701606733
4100
83.8255547054323
4200
84.20811017597552
4300
84.52945677123182
4400
84.5294567712

wandb: Network error (ConnectionError), entering retry loop.


96000
73.89441469013008
96100
73.89441469013008
96200
73.89441469013008
96300
73.89441469013008
96400
73.89441469013008
96500
73.89441469013008
96600
73.89441469013008
96700
73.89441469013008
96800
73.89441469013008
96900
73.89441469013008
97000
73.89441469013008
97100
73.89441469013008
97200
73.89441469013008
97300
73.89441469013008
97400
73.89441469013008
97500
73.89441469013008
97600
73.89441469013008
97700
73.89441469013008
97800
73.89441469013008
97900
73.89441469013008
98000
73.89441469013008
98100
73.89441469013008
98200
73.89441469013008
98300
73.89441469013008
98400
73.89441469013008
98500
73.89441469013008
98600
73.89441469013008
98700
73.89441469013008
98800
73.89441469013008
98900
73.89441469013008
99000
73.89441469013008
99100
73.89441469013008
99200
73.89441469013008
99300
73.89441469013008
99400
73.89441469013008
99500
73.89441469013008
99600
73.89441469013008
99700
73.89441469013008
99800
73.89441469013008
99900
73.89441469013008
tensor(26.1056, device='cuda:0', grad_fn

In [217]:
y_pred.shape

torch.Size([6535, 1])

In [228]:
torch.round(y_pred[:10])

tensor([[1.],
        [1.],
        [0.],
        [1.],
        [1.],
        [1.],
        [0.],
        [1.],
        [1.],
        [1.]], device='cuda:0', grad_fn=<RoundBackward0>)

In [227]:
testy[:10]

tensor([0., 1., 1., 0., 1., 0., 1., 1., 1., 1.], device='cuda:0')

In [220]:
def predict(model, test_loader):
    all_preds = []
    all_preds_raw = []
    all_labels = []

    for batch in test_loader:
        batch.x = torch.tensor(batch.x)
        batch.x = batch.x.reshape((-1, *batch.x.shape[2:]))
        batch.to(device)  
        pred = model(torch.tensor(batch.x).float(), 
                        #batch.edge_attr.float(),
                        batch.edge_index, 
                        batch.batch) 

        all_preds.append(np.argmax(pred.cpu().detach().numpy(), axis=1))
        all_preds_raw.append(torch.sigmoid(pred).cpu().detach().numpy())
        all_labels.append(batch.y.cpu().detach().numpy())
    
    all_preds = np.concatenate(all_preds).ravel()
    all_labels = np.concatenate(all_labels).ravel()
    return all_preds, all_preds_raw, all_labels

In [232]:
torch.round(testX[0])

tensor([0., 0., 0., 1., 1., 1., 0., 1., 1., 1., 0., 0., 0., 0., 0., 0.],
       device='cuda:0')

In [229]:
testy

tensor([0., 1., 1.,  ..., 0., 0., 1.], device='cuda:0')

In [223]:
len(model(testX))

1401

In [237]:
model(testX)[0:10]

tensor([[0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [1.],
        [1.]], device='cuda:0', grad_fn=<SliceBackward0>)

In [240]:
testy[:10]

tensor([0., 1., 1., 0., 1., 0., 1., 1., 1., 1.], device='cuda:0')

In [235]:
lol = model(testX[20:30])
print(lol)
#print(lol.flatten())
print(testy[20:30])

tensor([[0.],
        [0.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [0.],
        [1.],
        [0.]], device='cuda:0', grad_fn=<SigmoidBackward0>)
tensor([0., 0., 0., 0., 0., 1., 0., 0., 1., 1.], device='cuda:0')


In [225]:
testy[:10]

tensor([0., 1., 1., 0., 1., 0., 1., 1., 1., 1.], device='cuda:0')

In [226]:
predict(model, test)

AttributeError: 'str' object has no attribute 'x'

In [ ]:
import pandas as pd
import matplotlib
from matplotlib import pyplot as plt
import seaborn as sns
import tempfile
import os

In [ ]:
import tensorflow as tf